In [ ]:
!git clone https://github.com/kexinhuang12345/clinicalBERT.git

In [2]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 123 kB 5.4 MB/s 
     |████████████████████████████████| 131 kB 28.9 MB/s 
     |████████████████████████████████| 8.1 MB 38.4 MB/s 
     |████████████████████████████████| 79 kB 6.6 MB/s 
     |████████████████████████████████| 138 kB 58.1 MB/s 
     |████████████████████████████████| 127 kB 59.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/drive')atplotlib

Mounted at /content/drive


In [4]:
!python /content/clinicalBERT/preprocess.py

tcmalloc: large alloc 1073741824 bytes == 0x55edc6afc000 @  0x7fcaacbaa2a4 0x7fca949cf9f5 0x7fca949d0d42 0x7fca949d245e 0x7fca949a4068 0x7fca949a3510 0x7fca9499ef4e 0x7fca94983fe4 0x55ed803394b0 0x55ed8042ae1d 0x55ed803ace99 0x55ed803a7ced 0x55ed8033abda 0x55ed803a8c0d 0x55ed803a79ee 0x55ed8033abda 0x55ed803a8c0d 0x55ed8033aafa 0x55ed803a8915 0x55ed803a79ee 0x55ed8033abda 0x55ed803acd00 0x55ed803a79ee 0x55ed803a76f3 0x55ed804714c2 0x55ed8047183d 0x55ed804716e6 0x55ed80449163 0x55ed80448e0c 0x7fcaab993bf7 0x55ed80448cea
sys:1: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
/content/clinicalBERT/preprocess.py:38: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_adm_notes.ADMITTIME_C = df_adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])
100% 43880/43880 [42:14<00:00, 17.31it/s]
100% 44112/44112 [1:07:08<00:00,

In [9]:
pip install funcsigs

In [ ]:
!python /content/clinicalBERT/run_readmission.py --task_name readmission --readmission_mode early --do_eval --data_dir /content/clinicalBERT/data/3days --bert_model /content/clinicalBERT/model/ --max_seq_length 512 --output_dir /content/clinicalBERT/result

in the modeling class
11/07/2021 00:30:45 - INFO - __main__ -   device cpu n_gpu 0 distributed training False
11/07/2021 00:30:45 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
11/07/2021 00:30:45 - INFO - modeling_readmission -   loading archive file /content/clinicalBERT/model/
11/07/2021 00:30:45 - INFO - modeling_readmission -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

11/07/2021 00:30:49 - INFO - __main__ -   *** Example **

## Preprocess

In [ ]:
adm = pd.read_csv('/content/sample_data/ADMISSIONS.csv')

adm = adm.sort_values(['SUBJECT_ID','ADMITTIME'])
adm = adm.reset_index(drop = True)
adm['NEXT_ADMITTIME'] = adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)

adm['NEXT_ADMISSION_TYPE'] = adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)
rows = adm.NEXT_ADMISSION_TYPE == 'ELECTIVE' #elective are unplanned admissions
adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN
adm = adm.sort_values(['SUBJECT_ID','ADMITTIME'])

adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
adm['DAYS_NEXT_ADMIT']=  (adm.NEXT_ADMITTIME - adm.DISCHTIME).dt.total_seconds()/(24*60*60)

notes = pd.read_csv("/content/drive/MyDrive/NOTEEVENTS.csv")

notes_dis_sum = notes.loc[df_notes.CATEGORY == 'Discharge summary']
notes_dis_sum_last = (notes_dis_sum.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()

adm_notes = pd.merge(adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME']],
                        notes_dis_sum_last[['SUBJECT_ID','HADM_ID','TEXT']], 
                        on = ['SUBJECT_ID','HADM_ID'],
                        how = 'left')

adm_notes.groupby('ADMISSION_TYPE').apply(lambda g: g.TEXT.isnull().sum())/adm_notes.groupby('ADMISSION_TYPE').size()
clean = adm_notes.loc[adm_notes.ADMISSION_TYPE != 'NEWBORN'].copy()
clean['OUTPUT_LABEL'] = (clean.DAYS_NEXT_ADMIT < 30).astype('int')

model